In [49]:
import tensorflow as tf
import numpy as np


# STEP 1: define the neural network topology
# define training batch size
batch_size = 8

# use seed to make sure every time got the same w1, w2
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

# use placeholder instead of constant to reduce resource consumption
# use None dimension for row to adjust the size of training dataset
x = tf.placeholder(tf.float32, shape=(None, 2), name='x-input')
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y-input')
# x = tf.constant([[0.7, 0.9]])

# network topology: x-(w1)-a-(w2)-y
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)


# STEP 2: define loss function and back propagation algorithm
y = tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(
    y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0))
    + (1-y_) * tf.log(tf.clip_by_value(1-y, 1e-10, 1.0)))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)


# STEP 3: define the session to execute
# generate a random test dataset
rdm = np.random.RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
# labeling 1 as positive, 0 as negative
Y = [[int(x1+x2 < 1)] for (x1,x2) in X]

with tf.Session() as sess:
    # use global variables initialization
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    # sess.run(w1.initializer)
    # sess.run(w2.initializer)
    
    print('before training weights:')
    print(sess.run(w1))
    print(sess.run(w2))
    
    # set training times
    STEPS = 5000
    for i in range(STEPS):
        # pick data with batch_size to train
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        
        # train neural network and adjust weights
        sess.run(train_step, feed_dict={x:X[start:end], y_:Y[start:end]})
        if i % 1000 == 0:
            total_cross_entropy = sess.run(cross_entropy, feed_dict={x:X, y_:Y})
            print('after %d training steps, cross entroy is %g' % (i, total_cross_entropy))
    
    print('after training weights:')
    print(sess.run(w1))
    print(sess.run(w2))

before training weights:
[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
after 0 training steps, cross entroy is 1.89805
after 1000 training steps, cross entroy is 0.655075
after 2000 training steps, cross entroy is 0.626172
after 3000 training steps, cross entroy is 0.615096
after 4000 training steps, cross entroy is 0.610309
after training weights:
[[ 0.02476983  0.56948674  1.6921941 ]
 [-2.1977348  -0.23668921  1.1143895 ]]
[[-0.45544702]
 [ 0.49110925]
 [-0.98110336]]
